In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('../Data/processed_data.csv')
test_data = pd.read_csv('../Data/test_data.csv')
train_data = pd.read_csv('../Data/train_data.csv')
val_data = pd.read_csv('../Data/val_data.csv')

In [3]:
df

,gene_id,transcript_id,transcript_position,Key,Value_1,Value_2,Value_3,Value_4,Value_5,Value_6,Value_7,Value_8,Value_9,transcript_id_encoded,Key_encoded,gene_id_encoded,label
0,ENSG00000169045,ENST00000514332,579,CAAACAA,0.007097,2.389750,104.571429,0.006643,2.526429,98.371429,0.005408,2.419286,89.453571,4374,72,2833,0
1,ENSG00000136891,ENST00000374902,2482,CTGACAG,0.007560,3.764643,106.964286,0.005346,8.643929,113.821429,0.007909,2.611071,80.214286,2968,134,1861,0
2,ENSG00000101057,ENST00000217026,1738,CAGACCC,0.009036,8.260000,108.836538,0.011480,5.427885,128.096154,0.005846,3.924615,79.675000,151,89,632,0
3,ENSG00000109332,ENST00000394803,1321,AGAACAG,0.008855,7.054000,128.133333,0.008535,5.120667,95.853333,0.006006,3.055000,88.960000,3364,26,953,0
4,ENSG00000128283,ENST00000249014,598,GGGACAC,0.009415,4.048154,117.128205,0.009710,7.149872,117.666667,0.007828,3.196282,82.721795,474,181,1534,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENSG00000170421,ENST00000552551,1370,GGAACAT,0.007304,8.752857,119.823954,0.008027,3.603102,97.095815,0.006014,2.908407,87.478066,4698,171,2887,0
121834,ENSG00000006125,ENST00000621914,5135,GAAACAA,0.009418,3.244868,104.998684,0.007654,3.598947,98.059211,0.005389,3.381447,85.511842,5244,144,31,0
121835,ENSG00000174695,ENST00000511450,415,TTAACTC,0.007323,1.770944,93.327778,0.006851,1.988611,98.191667,0.007017,1.543528,92.108333,4358,273,3018,0
121836,ENSG00000072849,ENST00000158771,1013,GAAACAA,0.011057,3.060000,105.620000,0.006312,2.690667,97.173333,0.006495,2.360167,89.703333,38,144,295,0


In [4]:
X_train = train_data.drop(columns=['label','gene_id', 'Key', 'transcript_id'])
y_train = train_data['label']

X_val = val_data.drop(columns=['label','gene_id', 'Key', 'transcript_id'])
y_val = val_data['label']

X_test = test_data.drop(columns=['label','gene_id', 'Key', 'transcript_id'])
y_test = test_data['label']

In [5]:
X_train

,transcript_position,Value_1,Value_2,Value_3,Value_4,Value_5,Value_6,Value_7,Value_8,Value_9,transcript_id_encoded,Key_encoded,gene_id_encoded
0,579,0.007097,2.389750,104.571429,0.006643,2.526429,98.371429,0.005408,2.419286,89.453571,4374,72,2833
1,1738,0.009036,8.260000,108.836538,0.011480,5.427885,128.096154,0.005846,3.924615,79.675000,151,89,632
2,1321,0.008855,7.054000,128.133333,0.008535,5.120667,95.853333,0.006006,3.055000,88.960000,3364,26,953
3,598,0.009415,4.048154,117.128205,0.009710,7.149872,117.666667,0.007828,3.196282,82.721795,474,181,1534
4,839,0.008583,1.840789,90.740000,0.008351,1.756944,92.806667,0.008282,2.141222,87.446667,5192,120,3792
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85766,4650,0.007948,2.477727,110.318182,0.007562,2.891364,106.327273,0.004942,2.267727,95.336364,251,11,23
85767,645,0.007640,9.002093,128.046512,0.008016,2.914264,96.216279,0.006287,2.122233,89.341085,2163,24,275
85768,111,0.008148,6.488605,121.581395,0.008446,3.957442,95.839535,0.006345,2.230233,87.481395,3037,170,3219
85769,1370,0.007304,8.752857,119.823954,0.008027,3.603102,97.095815,0.006014,2.908407,87.478066,4698,171,2887


In [6]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [7]:
#valid set
val_probabilities = model.predict_proba(X_val)[:, 1]

val_predictions = model.predict(X_val)

val_roc_auc = roc_auc_score(y_val, val_probabilities)

val_accuracy = accuracy_score(y_val, val_predictions)

print(f'Validation ROC AUC Score: {val_roc_auc:.2f}')
print(f'Validation Accuracy Score: {val_accuracy:.2f}')

Validation ROC AUC Score: 0.67
Validation Accuracy Score: 0.95


ROC AUC is low might be due to class imbalance

In [8]:
#test set
test_predictions = model.predict(X_test)
test_probabilities = model.predict_proba(X_test)[:, 1]
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Test Accuracy: {test_accuracy}")
test_roc_auc = roc_auc_score(y_test, test_probabilities)
print(f"Test ROC AUC: {test_roc_auc}")

Test Accuracy: 0.9544396408380446
Test ROC AUC: 0.6731633843971391


In [9]:
model2 = LogisticRegression(class_weight='balanced', max_iter=1000)
model2.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [10]:
#valid set
val_probabilities = model2.predict_proba(X_val)[:, 1]

val_predictions = model2.predict(X_val)

val_roc_auc = roc_auc_score(y_val, val_probabilities)

val_accuracy = accuracy_score(y_val, val_predictions)

print(f'Validation ROC AUC Score: {val_roc_auc:.2f}')
print(f'Validation Accuracy Score: {val_accuracy:.2f}')

Validation ROC AUC Score: 0.69
Validation Accuracy Score: 0.61


In [11]:
#test set
test_predictions = model2.predict(X_test)
test_probabilities = model2.predict_proba(X_test)[:, 1]
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Test Accuracy: {test_accuracy}")
test_roc_auc = roc_auc_score(y_test, test_probabilities)
print(f"Test ROC AUC: {test_roc_auc}")

Test Accuracy: 0.6108524553818867
Test ROC AUC: 0.687844299193774


In [12]:
smote = SMOTE(random_state=42)
X_train2, y_train2 = smote.fit_resample(X_train, y_train)
model3 = LogisticRegression(class_weight='balanced', max_iter=1000)
model3.fit(X_train2, y_train2)

c:\Users\lakbo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', max_iter=1000)

In [13]:
#valid set
val_probabilities = model3.predict_proba(X_val)[:, 1]

val_predictions = model3.predict(X_val)

val_roc_auc = roc_auc_score(y_val, val_probabilities)

val_accuracy = accuracy_score(y_val, val_predictions)

print(f'Validation ROC AUC Score: {val_roc_auc:.2f}')
print(f'Validation Accuracy Score: {val_accuracy:.2f}')

Validation ROC AUC Score: 0.69
Validation Accuracy Score: 0.61


In [14]:
#test set
test_predictions = model3.predict(X_test)
test_probabilities = model3.predict_proba(X_test)[:, 1]
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Test Accuracy: {test_accuracy}")
test_roc_auc = roc_auc_score(y_test, test_probabilities)
print(f"Test ROC AUC: {test_roc_auc}")

Test Accuracy: 0.6133466356279792
Test ROC AUC: 0.6860519087464076


In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
model4 = LogisticRegression(class_weight='balanced', max_iter=1000)
model4.fit(X_train_resampled, y_train_resampled)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [16]:
val_predictions = model4.predict(X_val_scaled)
val_probabilities = model4.predict_proba(X_val_scaled)[:, 1]
val_accuracy = accuracy_score(y_val, val_predictions)
val_roc_auc = roc_auc_score(y_val, val_probabilities)

print(f'Validation ROC AUC Score: {val_roc_auc:.2f}')
print(f'Validation Accuracy Score: {val_accuracy:.2f}')

Validation ROC AUC Score: 0.73
Validation Accuracy Score: 0.64


In [17]:
test_predictions = model4.predict(X_test_scaled)
test_probabilities = model4.predict_proba(X_test_scaled)[:, 1]
test_accuracy = accuracy_score(y_test, test_predictions)
test_roc_auc = roc_auc_score(y_test, test_probabilities)

print(f'Test Accuracy: {test_accuracy:.2f}')
print(f'Test ROC AUC: {test_roc_auc:.2f}')

Test Accuracy: 0.66
Test ROC AUC: 0.72
